In [10]:
import pandas as pd

In [27]:

df_label = pd.read_csv('./train/label', header=None)
print(df_label)
df_sentence = pd.read_csv('./train/seq.in', header=None)
df_tags = pd.read_csv('./train/seq.out', header=None)

df = pd.concat([df_label, df_sentence, df_tags], axis=1)

df.columns = ['intent', 'sentence', 'entities']

df.to_json('./train.json', orient='records')

                        0
0               PlayMusic
1           AddToPlaylist
2                RateBook
3               PlayMusic
4           AddToPlaylist
...                   ...
13079      BookRestaurant
13080           PlayMusic
13081            RateBook
13082  SearchCreativeWork
13083            RateBook

[13084 rows x 1 columns]


In [28]:

df_label = pd.read_csv('./test/label', header=None)
print(df_label)
df_sentence = pd.read_csv('./test/seq.in', header=None)
df_tags = pd.read_csv('./test/seq.out', header=None)

df = pd.concat([df_label, df_sentence, df_tags], axis=1)

df.columns = ['intent', 'sentence', 'entities']

df.to_json('./test.json', orient='records')

                      0
0         AddToPlaylist
1        BookRestaurant
2         AddToPlaylist
3            GetWeather
4             PlayMusic
..                  ...
695  SearchCreativeWork
696  SearchCreativeWork
697            RateBook
698       AddToPlaylist
699            RateBook

[700 rows x 1 columns]


In [29]:
import pandas as pd

df = pd.read_json('./test.json')
df

,intent,sentence,entities
0,AddToPlaylist,add sabrina salerno to the grime instrumentals...,O B-artist I-artist O O B-playlist I-playlist O
1,BookRestaurant,i want to bring four people to a place that s ...,O O O O B-party_size_number O O O O O O B-spat...
2,AddToPlaylist,put lindsey cardinale into my hillary clinton ...,O B-artist I-artist O B-playlist_owner B-playl...
3,GetWeather,will it snow in mt on june 13 2038,O O B-condition_description O B-state O B-time...
4,PlayMusic,play signe anderson chant music that is newest,O B-artist I-artist B-music_item O O O B-sort
...,...,...,...
695,SearchCreativeWork,find a tv series called armageddon summer,O O B-object_type I-object_type O B-object_nam...
696,SearchCreativeWork,find politicsnation with al sharpton,O B-object_name I-object_name I-object_name I-...
697,RateBook,rate this album 0 points out of 6,O B-object_select B-object_type B-rating_value...
698,AddToPlaylist,add leah kauffman to my uncharted 4 nathan dra...,O B-artist I-artist O B-playlist_owner B-playl...


In [31]:
df = pd.read_csv('./intent_label.csv')
df

,UNK
0,AddToPlaylist
1,BookRestaurant
2,GetWeather
3,PlayMusic
4,RateBook
5,SearchCreativeWork
6,SearchScreeningEvent


In [16]:
from transformers import AutoTokenizer, BertModel, BertConfig
import pandas as pd

model_name = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

df = pd.read_json('./test_t.json')
tz = tokenizer(df['sentence'].to_numpy().tolist())

invalid_idx = []

for idx in range(len(df)):
    if len(df['sentence'][idx].split()) <= max([v for v in tz.word_ids(idx) if v != None]):
        invalid_idx.append(idx)
        
df = df.drop(index=invalid_idx)
df = df.reset_index(drop=True)


In [17]:


type_labels = pd.read_csv('./type_labels.csv').iloc[:,0].to_numpy().tolist()

def entity_to_list(entry):
    
    sentence_len = len(entry['sentence'].split())
    entities = entry['entities'].split()

    
    ret = [type_labels.index(tag) for tag in entities]
              
    return ret
        

df['entities'] = df.apply(entity_to_list, axis=1)

#df.iloc[-1000:].to_json('./valid.json', orient='records')
#df.iloc[:-1000].to_json('./train.json', orient='records')
df.to_json('./test.json', orient='records')


In [22]:
import pandas as pd

df = pd.read_json('./valid.json')
df

,intent,sentence,entities
0,SearchScreeningEvent,i d like to see a movies at the nearest movie...,"[1, 1, 1, 1, 1, 1, 29, 1, 1, 64, 33, 34]"
1,SearchScreeningEvent,find the colour of the chameleon,"[1, 27, 28, 28, 28, 28]"
2,SearchScreeningEvent,which movies are playing at the nearest cinema...,"[1, 29, 1, 1, 1, 1, 64, 33, 1, 68, 69]"
3,RateBook,the execution of justice only deserves 2 points,"[35, 36, 36, 36, 1, 1, 53, 52]"
4,RateBook,rate the republic of thieves chronicle three o...,"[1, 35, 36, 36, 36, 37, 53, 1, 1, 6, 52]"
...,...,...,...
995,BookRestaurant,i want to eat choucroute at a brasserie for 8,"[1, 1, 1, 1, 58, 1, 1, 56, 1, 45]"
996,PlayMusic,play funky heavy bluesy,"[1, 46, 47, 47]"
997,RateBook,rate the current album 2 points out of 6,"[1, 1, 39, 41, 53, 52, 1, 1, 6]"
998,SearchCreativeWork,go to the photograph the inflated tear,"[1, 1, 1, 41, 35, 36, 36]"


In [24]:
df = pd.read_json('./test.json')
df

,intent,sentence,entities
0,AddToPlaylist,add sabrina salerno to the grime instrumentals...,"[1, 4, 5, 1, 1, 46, 47, 1]"
1,BookRestaurant,i want to bring four people to a place that s ...,"[1, 1, 1, 1, 45, 1, 1, 1, 1, 1, 1, 64, 1, 50, ..."
2,AddToPlaylist,put lindsey cardinale into my hillary clinton ...,"[1, 4, 5, 1, 48, 46, 47, 47, 47, 47, 47, 47, 47]"
3,GetWeather,will it snow in mt on june 13 2038,"[1, 1, 9, 1, 66, 1, 68, 69, 69]"
4,PlayMusic,play signe anderson chant music that is newest,"[1, 4, 5, 31, 1, 1, 1, 62]"
...,...,...,...
649,SearchCreativeWork,find a tv series called armageddon summer,"[1, 1, 41, 42, 1, 35, 36]"
650,SearchCreativeWork,find politicsnation with al sharpton,"[1, 35, 36, 36, 36]"
651,RateBook,rate this album 0 points out of 6,"[1, 39, 41, 53, 52, 1, 1, 6]"
652,AddToPlaylist,add leah kauffman to my uncharted 4 nathan dra...,"[1, 4, 5, 1, 48, 46, 47, 47, 47, 1]"


In [2]:
import pandas as pd
df = pd.read_csv('./type_labels.csv')
df

,PAD
0,UNK
1,O
2,B-album
3,I-album
4,B-artist
...,...
68,B-timeRange
69,I-timeRange
70,B-track
71,I-track
